# 主题：基于随机森林的全基因组预测 
# 时间：2023-06-10

## 一、 利用随机森林（Random forest）实现从基因组中预测性状表型值

In [123]:
# scikit-learn是一个第三方库，需要安装
# !pip install --upgrade scikit-learn
# !conda update scikit-learn

In [121]:
## 导入常用基本库
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re, pip, conda

In [122]:
## 查看各个模块的版本
for package in [sklearn,np,pd,sns,pip,conda]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.2
numpy 1.21.5
pandas 1.4.4
seaborn 0.11.2
pip 22.2.2
conda 22.9.0


In [124]:
import scipy
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_validate,cross_val_predict,KFold

In [125]:
#导入基因数据
X = pd.read_csv(r"./Geno.csv")

In [126]:
X.head()

,ID,wPt.0538,wPt.8463,wPt.6348,wPt.9992,wPt.2838,wPt.8266,wPt.1100,wPt.0653,wPt.4418,...,c.408290,c.408294,c.408330,c.408336,c.408375,c.408393,c.408422,c.408424,c.408426,c.408443
0,775,0,1,1,1,1,0,1,1,1,...,1,0,1,1,1,0,0,0,0,1
1,2166,1,1,1,1,1,0,1,1,0,...,1,1,0,0,0,0,1,1,1,1
2,2167,1,1,1,1,1,0,1,1,0,...,1,1,0,0,0,0,1,1,1,1
3,2465,0,1,1,1,1,0,1,1,1,...,1,1,1,1,0,0,1,0,1,0
4,3881,0,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,0,0,1,1


In [127]:
X.columns.tolist()

['ID',
 'wPt.0538',
 'wPt.8463',
 'wPt.6348',
 'wPt.9992',
 'wPt.2838',
 'wPt.8266',
 'wPt.1100',
 'wPt.0653',
 'wPt.4418',
 'wPt.2152',
 'wPt.9049',
 'wPt.7068',
 'wPt.1681',
 'wPt.5480',
 'wPt.1695',
 'wPt.4025',
 'wPt.4125',
 'wPt.7238',
 'wPt.7924',
 'wPt.4029',
 'wPt.5672',
 'wPt.4600',
 'wPt.4209',
 'wPt.6777',
 'wPt.8124',
 'wPt.6117',
 'wPt.5877',
 'wPt.4814',
 'wPt.9368',
 'wPt.0245',
 'wPt.8894',
 'wPt.0473',
 'wPt.4602',
 'wPt.9196',
 'wPt.1741',
 'wPt.3855',
 'wPt.8921',
 'wPt.1505',
 'wPt.9532',
 'wPt.0205',
 'wPt.1547',
 'wPt.4427',
 'wPt.4419',
 'wPt.6853',
 'wPt.5027',
 'wPt.3611',
 'wPt.2866',
 'wPt.3605',
 'wPt.1272',
 'wPt.1940',
 'wPt.5688',
 'wPt.3533',
 'wPt.3563',
 'wPt.1191',
 'wPt.3677',
 'wPt.3774',
 'wPt.6214',
 'wPt.9270',
 'wPt.8015',
 'wPt.3109',
 'wPt.0789',
 'wPt.8021',
 'wPt.3804',
 'wPt.6343',
 'wPt.0312',
 'wPt.6965',
 'wPt.0049',
 'wPt.3879',
 'wPt.4720',
 'wPt.7063',
 'wPt.7229',
 'wPt.9757',
 'wPt.4283',
 'wPt.2185',
 'wPt.5234',
 'wPt.3451',
 'wPt

In [128]:
X.shape

(599, 1280)

In [129]:
#导入表型数据
Pheno = pd.read_csv("./Pheno.csv")

In [130]:
Pheno.head()

,ID,trait1,trait2,trait3,trait4
0,775,1.671629,-1.727470,-1.890285,0.050916
1,2166,-0.252703,0.409522,0.309386,-1.738759
2,2167,0.341815,-0.648626,-0.799559,-1.053569
3,2465,0.785439,0.093949,0.570468,0.551757
4,3881,0.998318,-0.282481,1.618682,-0.114285


In [131]:
Pheno.shape

(599, 5)

In [132]:
y = Pheno.iloc[:,1]

In [133]:
y

0      1.671629
1     -0.252703
2      0.341815
3      0.785439
4      0.998318
         ...   
594   -1.248504
595   -1.052126
596   -1.209974
597   -0.902397
598    0.222276
Name: trait1, Length: 599, dtype: float64

In [134]:
y.mean()

-3.338857988085525e-12

In [135]:
rfr =RFR(n_estimators = 100
        ,max_depth=15,random_state=2023) #定义随机森林训练器
cv = KFold(n_splits=10,shuffle=True,random_state=2023) #设置交叉验证

In [ ]:
# 自定义评价指标函数，计算皮尔逊相关系数
np.random.seed(2023)
def pearson_corr_score(y_true, y_pred):
    corr_coef, _ = scipy.stats.pearsonr(y_true, y_pred)
    return corr_coef

# 使用 cross_val_score 计算每一折的皮尔逊相关系数
scorer = make_scorer(pearson_corr_score)
result_rfr = cross_validate(rfr #要进行交叉验证的评估器
                          ,X,y #数据
                          ,cv=cv #交叉验证模式
                          ,scoring=scorer #评估指标
                          ,return_train_score=True #是否返回训练分数
                          ,verbose=True) #是否打印进程

In [ ]:
#显示预测结果
result_rfr 

In [138]:
trainPCC_rfr =result_rfr["train_score"]
testPCC_rfr = result_rfr["test_score"]

In [ ]:
#随机森林回归训练集十折平均
trainPCC_rfr.mean()

In [ ]:
#随机森林回归测试集十折平均结果
testPCC_rfr.mean()

In [ ]:
# 打印训练和测试在十折交叉过程中每一折的PCC值变化情况
xaxis = range(1,11)
plt.figure(figsize=(8,6),dpi=80)
# 绘制PCC
plt.plot(xaxis,trainPCC_rfr,color="#36ACA2",label = "RandomForestTrain")
plt.plot(xaxis,testPCC_rfr,color="#614099",linestyle="--",label = "RandomForestTest")
plt.xticks([1,2,3,4,5,6,7,8,9,10])
plt.xlabel("folds",fontsize=16)
plt.ylabel("PCC values",fontsize=16)
plt.legend()
plt.show()

In [111]:
# 使用 cross_val_predict 获取每个样本的预测值
np.random.seed(2023)
predicted_values = cross_val_predict(rfr #随机森林训练器
                                     ,X,y #基因型和表型数据集
                                     ,cv=cv) #定义的交叉验证

In [ ]:
# 打印每个样本的预测值
for idx, predicted_value in enumerate(predicted_values):
    print(f"sample {idx+1}'predicted value: {predicted_value}")

In [113]:
MSE = mean_squared_error(y, predicted_values)
RMSE = np.sqrt(mean_squared_error(y, predicted_values))
MAE = mean_absolute_error(y, predicted_values)
PCCs = pearsonr(y, predicted_values)

In [ ]:
print(f"MSE={MSE}")
print(f"RMSE={RMSE}")
print(f"MAE={MAE}")
print(f"PCCs={PCCs}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


# 计算预测值和真实值的核密度估计
kde_predicted = gaussian_kde(predicted_values)
kde_true = gaussian_kde(y)

# 设置绘图参数
color_predicted = 'blue'  # 预测值的颜色
color_true = 'red'  # 真实值的颜色
fit_line_color = 'green'  # 拟合线条颜色
error_band_color = 'gray'  # 误差带颜色

# 绘制核密度图
min_value = min(min(predicted_values), min(y))
max_value = max(max(predicted_values), max(y))
x = np.linspace(min_value, max_value, 1000)

fig, ax = plt.subplots()
ax.fill_between(x, kde_predicted(x), color=color_predicted, alpha=0.3)
ax.fill_between(x, kde_true(x), color=color_true, alpha=0.3)
ax.plot(x, kde_predicted(x), color=fit_line_color, label='Predicted values')
ax.plot(x, kde_true(x), color=fit_line_color, linestyle='dashed', label='Observed values')

# 设置图形标签和标题
ax.set_xlabel('Values')
ax.set_ylabel('Density')
ax.set_title('Predicted values vs Observed values Density')
ax.legend()

# 显示图形
plt.show()

In [ ]:
# 导入绘制三点密度图的第三方库
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# 定义预测和观察值
X = predicted_values
Y = y

# 计算散点密度
xy = np.vstack([X, Y])
z = gaussian_kde(xy)(xy)

# 按密度对点进行排序，密度最大的点最后绘制
idx = z.argsort()
X, Y, z = X[idx], Y[idx], z[idx]

fig, ax = plt.subplots()
plt.scatter(X, Y, c=z, s=10, cmap='viridis')
plt.colorbar()

# 添加1：1参考线
min_val = min(X.min(), Y.min())
max_val = max(X.max(), Y.max())
plt.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--')

# 计算线性拟合线，用于大致判断预测和观测值变化趋势
fit = np.polyfit(Y, X, 1)
fit_fn = np.poly1d(fit)
plt.plot(X, fit_fn(X), color='black')

# 显示拟合直线方程
equation = f'y = {fit_fn.coeffs[0]:.3f}x + {fit_fn.coeffs[1]:.3f}'
plt.text(0.05, 0.85, equation, transform=ax.transAxes, ha='left', va='top')

# 计算观察值和预测值之间PCC和RMSE值
rmse = np.sqrt(mean_squared_error(X, Y))
pcc, _ = pearsonr(X, Y)
plt.text(0.05, 0.95, f'RMSE: {rmse:.3f}\nPCC: {pcc:.3f}', transform=ax.transAxes, ha='left', va='top')

plt.xlabel('Observed Values')
plt.ylabel('Predicted Values')
plt.title('RF')
plt.show()


## 二、随机森林回归预测的参数设置及超参数优化

### 2.1 优化之前如何确定相应超参数的优化范围

- 通过打印随机什么的学习曲线来判断：

In [117]:
# 在第一部中，我们根据经验值设定了n_estimators的数量为100，在下面学习曲线的打印及优化过程中，我们将继承第一部分的全部随机种子
# 定义每迭代次数数组，初始为5，步长为5
i = [1,*range(5,101,5)]

In [142]:
#生成保存模型训练集，测试集标准差的结果数组
trainRMSE = np.array([])
testRMSE = np.array([])
trainSTD = np.array([])
testSTD = np.array([])

#在参数空间中迭代打印每次更新的结果
for n_estimators in i:
    
    #根据每次迭代的参数，定义训练器
    rfr = RFR(n_estimators=n_estimators,random_state=2023)
    
    #采用交叉验证进行全过程，并且打印十折交叉验证的结果
    cv = KFold(n_splits=10,shuffle=True,random_state=2023)
    result_rfr = cross_validate(rfr
                                ,X,y
                                ,cv=cv
                                ,scoring="neg_mean_squared_error"
                                ,return_train_score=True)
    
    # 计算RMSE值评价模型的性能
    train = abs(result_rfr["train_score"])**0.5
    test = abs(result_rfr["test_score"])**0.5
    
    #将每次迭代产生的结果保存在数组中
    trainRMSE = np.append(trainRMSE,train.mean()) 
    testRMSE = np.append(testRMSE,test.mean())
    trainSTD = np.append(trainSTD,train.std()) 
    testSTD = np.append(testSTD,test.std())

In [143]:
#定义一个绘图函数
def CVresult(i,trainRMSE,testRMSE,trainSTD,testSTD):

    xaxis = i
    plt.figure(figsize=(8,6),dpi=80)

    #RMSE
    plt.plot(xaxis,trainRMSE,color="k",label = "RandomForestTrain")
    plt.plot(xaxis,testRMSE,color="red",label = "RandomForestTest")

    #绘制标准差和RMSE图
    plt.plot(xaxis,trainRMSE+trainSTD,color="k",linestyle="dotted")
    plt.plot(xaxis,trainRMSE-trainSTD,color="k",linestyle="dotted")
    plt.plot(xaxis,testRMSE+testSTD,color="#36ACA2",linestyle="dotted")
    plt.plot(xaxis,testRMSE-testSTD,color="#614099",linestyle="dotted")
    plt.xticks([*xaxis])
    plt.legend(loc=1)
    plt.show()

In [ ]:
CVresult(i,trainRMSE,testRMSE,trainSTD,testSTD)

**从上图来看，当随机森林数量达到100时，训练集和测试集的RMSE趋于平缓，因此可以大致确定n_estimators的范围在100左右。**

### 2.2 采用网格搜索方法进行超参数优化

In [145]:
# 导入相关模块
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import time #计时模块time
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import cross_validate, KFold, GridSearchCV

In [146]:
# 导入基因型数据
X = pd.read_csv(r"./Geno.csv")

In [147]:
# 导入性状表型数据
Pheno = pd.read_csv("./Pheno.csv")
y = Pheno.iloc[:,1]

In [163]:
def RMSE(cvresult,key):
    return (abs(cvresult[key])**0.5).mean()

In [164]:
reg = RFR(random_state=2023)
cv = KFold(n_splits=10,shuffle=True,random_state=2023)

In [ ]:
result_pre_adjusted = cross_validate(reg
                                     ,X,y
                                     ,cv=cv
                                     ,scoring="neg_mean_squared_error"
                                     ,return_train_score=True
                                     ,verbose=True)

In [ ]:
RMSE(result_pre_adjusted,"train_score")

In [ ]:
RMSE(result_pre_adjusted,"test_score")

In [153]:
# 定义一个超参数优化空间，选择需要优化的重要超参数

param_grid_simple = {'n_estimators': [*range(20,200,5)]
                     , 'max_depth': [*range(10,25,2)]
                     , "max_features": ["log2","sqrt","auto"]
                     , "min_impurity_decrease": [*np.arange(0,5,10)]
                    }

In [154]:
# 定义评估器、交叉验证评估器与网格搜索评估器

reg = RFR(random_state=2023,verbose=True)
cv = KFold(n_splits=10,shuffle=True,random_state=2023)
search = GridSearchCV(estimator=reg
                     ,param_grid=param_grid_simple
                     ,scoring = "neg_mean_squared_error"
                     ,verbose = True
                     ,cv = cv)

In [155]:
# 训练评估器，此过程有点费时：

start = time.time()
search.fit(X,y)
print(time.time() - start)

Fitting 10 folds for each of 864 candidates, totalling 8640 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

...

[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    6.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 170 out of 170 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    7.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 185 out of 185 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done 195 out of 195 | elapsed:    7.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 195 out of 195 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


13137.255609750748


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:    6.0s finished


In [156]:
# 查看最优结果
search.best_estimator_

RandomForestRegressor(max_depth=18, min_impurity_decrease=0, n_estimators=135,
                      random_state=2023, verbose=True)

In [157]:
abs(search.best_score_)**0.5

0.7329691697972832

In [170]:
# 根据优化结果复现，并选择性保留需要的结果
adj_rfr = RFR(n_estimators=135, max_depth=18, random_state=2023)

In [171]:
np.random.seed(2023)
cv = KFold(n_splits=10,shuffle=True,random_state=2023)
predicted_adjusted = cross_val_predict(adj_rfr
                                      ,X,y
                                      ,cv=cv)

In [172]:
MSE = mean_squared_error(y, predicted_adjusted)
RMSE = np.sqrt(mean_squared_error(y, predicted_adjusted))
MAE = mean_absolute_error(y, predicted_adjusted)
PCCs = pearsonr(y, predicted_adjusted)

In [173]:
print(f"MSE={MSE}")
print(f"RMSE={RMSE}")
print(f"MAE={MAE}")
print(f"PCCs={PCCs}")

MSE=0.5373886528037769
RMSE=0.7330679728400205
MAE=0.5749577801447862
PCCs=PearsonRResult(statistic=0.6802744277408996, pvalue=1.3560952997800532e-82)


### 2.3 利用bayes-opt对RF进行超参数优化

In [1]:
# 导入常用的库
import time
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt

#导入优化算法
import bayes_opt
from scipy.stats import pearsonr
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_validate,cross_val_predict,KFold,GridSearchCV

In [2]:
# 导入基因型数据
G = pd.read_csv(r"./Geno.csv")

In [3]:
# 导入表型数据
Pheno = pd.read_csv(r"./Pheno.csv")
P = Pheno.iloc[:,1]

In [9]:
def Bayesopt_objective(n_estimators,max_depth):
    
    #定义评估器
    reg = RFR(n_estimators = int(n_estimators)
              ,max_depth = int(max_depth)
              ,max_features = "sqrt"
              ,random_state=2023)
    
    cv = KFold(n_splits=10,shuffle=True,random_state=2023)
    validation_loss = cross_validate(reg,G,P
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,error_score='raise'
                                     #如果交叉验证中的算法执行报错，则告诉我们错误的理由
                                    )
    
    #交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    #目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

In [10]:
param_space = {"n_estimators": (20,200)
               , "max_depth":(5,25)
                }

In [11]:
def Bayes_param_opt(init_points,n_iter):
    
    #定义优化器
    opt = BayesianOptimization(Bayesopt_objective #需要优化的目标函数
                               ,param_space #备选参数空间
                               ,random_state=2023 #随机数种子，虽然无法控制住
                              )
    
    #使用优化器，记住bayes_opt只支持最大化
    opt.maximize(init_points = init_points #抽取多少个初始观测值
                 , n_iter=n_iter #一共观测/迭代多少次
                )
    
    #优化完成，取出最佳参数与最佳分数
    params_best = opt.max["params"]
    score_best = opt.max["target"]
    
    #打印最佳参数与最佳分数
    print("\n","\n","best params: ", params_best,
          "\n","\n","best cvscore: ", score_best)
    
    #返回最佳参数与最佳分数
    return params_best, score_best

In [ ]:
start = time.time()
params_best, score_best = Bayes_param_opt(20,280) #初始看20个观测值，后面迭代280次
print('It takes %s minutes' % ((time.time() - start)/60))

### ※重新构造超参数空间

上述优化过程中，结果是在十折交叉验证下进行的，结合第一部分默认参数和第一次优化的结果来看，从测试集的RMSE结果可能存在过拟合现象。

In [21]:
def Bayesopt_objective(n_estimators,max_depth):
    
    #定义评估器
    reg = RFR(n_estimators = int(n_estimators)
              ,max_depth = int(max_depth)
              ,random_state = 2023)
    
    cv = KFold(n_splits=10,shuffle=True,random_state=2023)
    validation_loss = cross_validate(reg,G,P
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,error_score='raise'#如果交叉验证中的算法执行报错，则告诉我们错误的理由
                                    )
    
    #交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    #目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

In [22]:
param_space = {"n_estimators": (20,200)
               , "max_depth":(5,25)
                }

In [23]:
def Bayes_param_opt(init_points,n_iter):
    
    #定义优化器
    opt = BayesianOptimization(Bayesopt_objective #需要优化的目标函数
                               ,param_space #备选参数空间
                               ,random_state=2023 #随机数种子，虽然无法控制住
                              )
    
    #使用优化器，记住bayes_opt只支持最大化
    opt.maximize(init_points = init_points #抽取多少个初始观测值
                 , n_iter=n_iter #一共观测/迭代多少次
                )
    
    #优化完成，输出最佳参数与最佳评分
    params_best = opt.max["params"]
    score_best = opt.max["target"]
    
    #打印最佳参数与最佳评分
    print("\n","\n","best params: ", params_best,
          "\n","\n","best cvscore: ", score_best)
    
    #返回最佳参数与最佳评分
    return params_best, score_best

In [ ]:
start = time.time()
params_best, score_best = Bayes_param_opt(20,280) #初始看20个观测值，后面迭代280次
print('It takes %s minutes' % ((time.time() - start)/60))

### ※超参数验证

In [27]:
def Bayes_opt_validation(params_best):
    
    reg = RFR(n_estimators = int(params_best["n_estimators"]) 
              ,max_depth = int(params_best["max_depth"])
              ,random_state=2023)

    cv = KFold(n_splits=10,shuffle=True,random_state=2023)
    validation_score = cross_validate(reg
                                          ,G
                                          ,P
                                          ,scoring="neg_root_mean_squared_error"
                                          ,cv=cv)
    return np.mean(validation_score["test_score"])

In [28]:
validation_result = abs(Bayes_opt_validation(params_best))
print("\n","\n","validation_result: ",validation_result)


 
 validation_result:  0.7296124291447439


In [33]:
def Bayes_opt_validation(params_best):
    
    reg = RFR(n_estimators = int(params_best["n_estimators"]) 
              ,max_depth = int(params_best["max_depth"])
              ,random_state=2023)

    cv = KFold(n_splits=10,shuffle=True,random_state=2023)
    validation_result = cross_val_predict(reg
                                          ,G
                                          ,P
                                          ,cv=cv)    
    return validation_result

In [34]:
#超参数优化结果
validation_result = Bayes_opt_validation(params_best)

In [ ]:
#显示优化结构
validation_result

In [36]:
MSE = mean_squared_error(P, validation_result)
RMSE = np.sqrt(mean_squared_error(P, validation_result))
MAE = mean_absolute_error(P, validation_result)
PCCs = pearsonr(P, validation_result)

In [ ]:
print(f"MSE={MSE}")
print(f"RMSE={RMSE}")
print(f"MAE={MAE}")
print(f"PCCs={PCCs}")

### ※可视化
- **预测值和观测值的密度分布图**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


# 计算预测值和真实值的核密度估计
kde_predicted = gaussian_kde(validation_result)
kde_true = gaussian_kde(P)

# 设置绘图参数
color_predicted = 'blue'  # 预测值的颜色
color_true = 'red'  # 真实值的颜色
fit_line_color = 'green'  # 拟合线条颜色
error_band_color = 'gray'  # 误差带颜色

# 绘制核密度图
min_value = min(min(validation_result), min(P))
max_value = max(max(validation_result), max(P))
x = np.linspace(min_value, max_value, 1000)

fig, ax = plt.subplots()
ax.fill_between(x, kde_predicted(x), color=color_predicted, alpha=0.3)
ax.fill_between(x, kde_true(x), color=color_true, alpha=0.3)
ax.plot(x, kde_predicted(x), color=fit_line_color, label='Predicted values')
ax.plot(x, kde_true(x), color=fit_line_color, linestyle='dashed', label='Observed values')

# 设置图形标签和标题
ax.set_xlabel('Values')
ax.set_ylabel('Density')
ax.set_title('Predicted values vs Observed values Density')
ax.legend()

# 显示图形
plt.show()

- **预测值和真实值的散点密度图**

In [ ]:
# 导入绘制三点密度图的第三方库
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# 定义预测和观察值
X1 = validation_result
Y = P

# 计算散点密度
xy = np.vstack([X1, Y])
z = gaussian_kde(xy)(xy)

# 按密度对点进行排序，密度最大的点最后绘制
idx = z.argsort()
X1, Y, z = X1[idx], Y[idx], z[idx]

fig, ax = plt.subplots()
plt.scatter(X1, Y, c=z, s=10, cmap='viridis')
plt.colorbar()

# 添加1：1参考线
min_val = min(X1.min(), Y.min())
max_val = max(X1.max(), Y.max())
plt.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--')

# 计算线性拟合线，用于大致判断预测和观测值变化趋势
fit = np.polyfit(Y, X1, 1)
fit_fn = np.poly1d(fit)
plt.plot(X1, fit_fn(X1), color='black')

# 显示拟合直线方程
equation = f'y = {fit_fn.coeffs[0]:.3f}x + {fit_fn.coeffs[1]:.3f}'
plt.text(0.05, 0.85, equation, transform=ax.transAxes, ha='left', va='top')

# 计算观察值和预测值之间PCC和RMSE值
rmse = np.sqrt(mean_squared_error(X1, Y))
pcc, _ = pearsonr(X1, Y)
plt.text(0.05, 0.95, f'RMSE: {rmse:.3f}\nPCC: {pcc:.3f}', transform=ax.transAxes, ha='left', va='top')

plt.xlabel('Observed Values')
plt.ylabel('Predicted Values')
plt.title('RF')
plt.show()